# PyTorch로 구현하는 다층 퍼셉트론 (MLP)

**학습 목표:**
- 대표적인 딥러닝 프레임워크인 **PyTorch**의 기본 구조와 사용법을 익힙니다.
- 딥러닝의 'Hello, World!'와 같은 **MNIST 손글씨 숫자** 데이터셋을 처리하는 방법을 배웁니다. (`Dataset`, `DataLoader`)
- `nn.Module`을 상속받아 **다층 퍼셉트론(MLP)** 모델을 직접 정의하고, 각 계층의 역할을 이해합니다.
- 손실 함수(Cross-Entropy), 옵티마이저(Adam)를 설정하고, 모델을 훈련시키는 전체 과정을 단계별로 학습합니다.
- 훈련 과정의 손실과 정확도를 시각화하여 모델의 학습 상태를 모니터링합니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

### (1) 하이퍼파라미터 및 장치 설정
모델 훈련에 필요한 주요 변수들을 설정합니다. GPU 사용이 가능할 경우, 연산 장치를 'cuda'로 설정하여 학습 속도를 높입니다.

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 하이퍼파라미터
input_size = 28 * 28  # MNIST 이미지 크기
hidden_size1 = 512
hidden_size2 = 256
num_classes = 10
learning_rate = 0.001
batch_size = 64
epochs = 10

Using device: cpu


### (2) 데이터 준비: MNIST
- `torchvision.transforms`를 사용하여 데이터를 텐서(Tensor)로 변환하고, 픽셀 값을 정규화하는 전처리 파이프라인을 구성합니다.
- `DataLoader`는 데이터를 미니배치(mini-batch) 단위로 묶고, 훈련 시 데이터를 섞어주는(shuffle) 역할을 수행하여 효율적인 학습을 돕습니다.

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(), # 이미지를 PyTorch 텐서로 변환
    transforms.Normalize((0.1307,), (0.3081,)) # MNIST 데이터의 평균과 표준편차로 정규화
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 데이터 확인
images, labels = next(iter(train_loader))
print(f"Images batch shape: {images.shape}")
print(f"Labels batch shape: {labels.shape}")

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:04<00:00, 2045310.18it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 139047.52it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:01<00:00, 1439144.60it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 3839284.31it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Images batch shape: torch.Size([64, 1, 28, 28])
Labels batch shape: torch.Size([64])


### (3) 모델 정의: MLP
`nn.Module`을 상속받아 사용자 정의 모델 클래스를 만듭니다. `__init__` 메서드에서 필요한 계층(layer)들을 정의하고, `forward` 메서드에서 데이터가 이 계층들을 통과하는 순서(순전파)를 정의합니다.

In [4]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size2, num_classes)

    def forward(self, x):
        # 입력 데이터를 1차원으로 평탄화
        x = x.view(-1, 28*28)
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)
        return out

model = MLP(input_size, hidden_size1, hidden_size2, num_classes).to(device)

### (4) 손실 함수 및 옵티마이저 설정
- **손실 함수 (Loss Function)**: 다중 클래스 분류 문제이므로 `nn.CrossEntropyLoss`를 사용합니다. 이 함수는 내부적으로 Softmax 함수를 적용한 후 Cross-Entropy를 계산하므로 모델의 마지막 계층에 활성화 함수를 추가할 필요가 없습니다.
- **옵티마이저 (Optimizer)**: 경사 하강법을 통해 모델의 가중치를 업데이트하는 역할을 합니다. 가장 널리 사용되는 `Adam`을 사용합니다.

In [5]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### (5) 모델 훈련
전체 훈련 데이터를 여러 번 반복하여 학습합니다(에포크). 각 에포크마다 `DataLoader`에서 미니배치를 가져와 다음 과정을 반복합니다.
1.  **순전파(Forward pass)**: 입력 데이터를 모델에 통과시켜 예측값을 얻습니다.
2.  **손실 계산**: 예측값과 실제 정답을 비교하여 손실을 계산합니다.
3.  **역전파(Backward pass)**: 손실을 최소화하기 위해 각 가중치에 대한 그래디언트(기울기)를 계산합니다 (`loss.backward()`).
4.  **가중치 업데이트**: 옵티마이저가 계산된 그래디언트를 사용하여 모델의 가중치를 업데이트합니다 (`optimizer.step()`).

In [6]:
for epoch in range(epochs):
    model.train() # 모델을 훈련 모드로 설정
    for batch_idx, (data, targets) in enumerate(train_loader):
        # 데이터를 설정한 장치로 이동
        data = data.to(device)
        targets = targets.to(device)
        
        # 순전파
        scores = model(data)
        loss = criterion(scores, targets)
        
        # 역전파 및 가중치 업데이트
        optimizer.zero_grad() # 그래디언트 초기화
        loss.backward()       # 그래디언트 계산
        optimizer.step()      # 가중치 업데이트
        
    # 에포크마다 결과 출력
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/10], Loss: 0.0210
Epoch [2/10], Loss: 0.0088
Epoch [3/10], Loss: 0.1561
Epoch [4/10], Loss: 0.1720
Epoch [5/10], Loss: 0.0227
Epoch [6/10], Loss: 0.0081
Epoch [7/10], Loss: 0.0026
Epoch [8/10], Loss: 0.0175
Epoch [9/10], Loss: 0.1459
Epoch [10/10], Loss: 0.0009


### (6) 모델 평가
학습이 완료된 모델의 성능을 테스트 데이터셋으로 평가합니다. 평가 시에는 불필요한 그래디언트 계산을 비활성화(`torch.no_grad()`)하여 메모리 사용량을 줄이고 계산 속도를 높입니다.

In [7]:
def check_accuracy(loader, model):
    model.eval() # 모델을 평가 모드로 설정
    num_correct = 0
    num_samples = 0
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device)
            
            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)
            
    accuracy = float(num_correct) / float(num_samples) * 100
    return accuracy

train_accuracy = check_accuracy(train_loader, model)
test_accuracy = check_accuracy(test_loader, model)

print(f'Accuracy on training set: {train_accuracy:.2f}%')
print(f'Accuracy on test set: {test_accuracy:.2f}%')

Accuracy on training set: 99.45%
Accuracy on test set: 97.98%
